#EURO2020
Sentiment analysis dei giocatori delle partite dell'Italia nella fase finale ad eliminazione dai quarti alla finale

* Scraping: streaming durante la partita BELITA ITAESP ITAEN
* Preprocessing
* ETL
* Analysis

##Import and path

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tweepy
import pandas as pd
pd.set_option('display.max_colwidth', None) # setta il num. di caratteri visibili per ogni cella della colonna
# es. 50 = primi 50 caratteri
# None = testo completo
import io
import os
import glob
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm

In [3]:
drive_dir = 'drive'
root = f'/content/{drive_dir}/MyDrive/'

root_main = f'{root}EURO2020/'
os.makedirs(root_main, exist_ok=True)

ingest_euro2020 = f'{root_main}ingest/'
os.makedirs(ingest_euro2020, exist_ok=True)

scrap_belita = f'{ingest_euro2020}scrap_belita/'
os.makedirs(scrap_belita, exist_ok=True)

scrap_itaesp = f'{ingest_euro2020}scrap_itaesp/'
os.makedirs(scrap_itaesp, exist_ok=True)

scrap_itaen = f'{ingest_euro2020}scrap_itaen/'
os.makedirs(scrap_itaen, exist_ok=True)

dwh_euro2020 = f'{root_main}dwh/'
os.makedirs(dwh_euro2020, exist_ok=True)

staging_area = f'{root_main}staging_area/'
os.makedirs(staging_area, exist_ok=True)

##Ingest (scraping)

In [ ]:
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    data = [] #qui ci metto i tweet che restano quando interrompo lo streaming
    num = 0 #questo è un contatore settato a 0

    def store(self):
      dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
      
      for status_j in self.data:
          status = status_j._json
          dict_['id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
          dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
          dict_['location'].append(status['user']['location'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
          dict_['date'].append(status['created_at'])
          dict_['text'].append(status['text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['retweet_count'].append(status['retweet_count'])
          dict_['retweet'].append(status['retweeted'])

      df = pd.DataFrame.from_dict(dict_, orient="columns")
      df = df.set_index("id")
      df.sort_values(by='favorite_count', inplace=True, ascending=False)
      # df.to_csv(f'{scrap_belita}sc_twitter_' + str(self.num) + '.csv') #salvo in csv BELITA
      # df.to_csv(f'{scrap_itaesp}sc_twitter_' + str(self.num) + '.csv') #salvo in csv ITAESP
      df.to_csv(f'{scrap_itaen}sc_twitter_' + str(self.num) + '.csv') #salvo in csv ITAEN
      self.num = self.num + 1
      self.data = []


    def on_status(self, status):
        self.data.append(status)
        # to pandas
        # store su google drive
        if len(self.data) > 10000:
          self.store()

In [ ]:
#api per tweepy
from tweepy import OAuthHandler
consumer_key = 'MQXChIn29dCky3pOQyW890Uxi'
consumer_secret = 'u4n3UeFz58b4H6InWFhBDUPZsfJ8xdVS0wODdnfAjz6e6PUbCJ'
access_token = '262247836-oWkm6AZIw4K0Fldnt8VBXwuL9IbtNCZwVj73ODkI'
access_secret = 'GySGl7BGaYvmXUtCapu5pHwEDHD5tA3Qd4yCMIDPKm1Wx'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

''' Altre chiavi
#api per tweepy
from tweepy import OAuthHandler
consumer_key = 'A4GRiWdUDgiGPckQSykzA'
consumer_secret = 'nSyjpFDTr3eEmd8tFeBM1lwVrTNPLaHrcx0DicTEEuw'
access_token = '1678747572-lbkg1jxpe5HypoTRSnRMeiVIe0RBMxhdjLJ0GJ6'
access_secret = '1qVP0pHXV9HDgKhHDIy2HHqqnHFBm3sI5AR2ZDwZxlM'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
'''

''' Altre chiavi
#api per tweepy
from tweepy import OAuthHandler
consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
'''

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [ ]:
#chiamata allo streaming
# myStream.filter(track=['belita' or 'euro2020' or 'belgioitalia' or 'italiabelgio']) #per BELITA
# myStream.filter(track=['ItaliaSpagna' or 'euro2020' or 'itaspa' or 'eurocopa2021' or 'EspanaItalia' or 'UefaEuro2020' or 'italiaespana' or 'ItalySpain']) #per ITAESP
myStream.filter(track=['Euro2020Final' or 'ITAENG' or 'EURO2020' or 'Wembley' or 'ENGITA' or 'ItalyEngland' or 'EnglandItaly' or 'InghilterraItalia' or 'ItaliaInghilterra']) #per ITAEN

KeyboardInterrupt: ignored

In [ ]:
#conto i tweet scaricati quando interrompo o si rompe
len(myStreamListener.data)

28

In [ ]:
# Create our dict with our data, come sopra per quando stoppo manualmente lo streaming
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in myStreamListener.data:
    status = status_j._json
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

In [ ]:
#creo il dataframe dell'ultimo stream stoppato manualmente
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(5)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1411091468199006209,RunawayAgain,Fri Jul 02 22:36:37 +0000 2021,👏🏻👏🏻👏🏻,0,[],"Roma, Lazio",False,0,1960,NaN,[]
1411092108602138629,naimrahman73,Fri Jul 02 22:39:10 +0000 2021,Full time at Piazza del Popolo 💚🤍♥️ #BELITA #I...,0,"[BELITA, ITA, ForzaAzzuri]",None,False,0,127,NaN,[]
1411092098539995136,yunginn_,Fri Jul 02 22:39:07 +0000 2021,RT @QueenMvy: MDMMDMDDRRR CE MEME PTNNN,0,[],SHE/HER,False,0,460,NaN,[QueenMvy]
1411092098712059907,HernamanAllen,Fri Jul 02 22:39:07 +0000 2021,"RT @BBCSport: ""It's pathetically amusing thoug...",0,[bbceuro2020],None,False,0,8,NaN,[BBCSport]
1411092098733031425,_1926Mattia,Fri Jul 02 22:39:07 +0000 2021,RT @azzurri: We move! 💚🤍❤️\n\n#ITA #BELITA #Az...,0,"[ITA, BELITA, Azzurri, VivoAzzurro]","Campania, Italia",False,0,36,NaN,[azzurri]
1411092099735470085,giorjolly,Fri Jul 02 22:39:08 +0000 2021,RT @r_esistancee: Praticamente a Caserta non s...,0,"[BELITA, BelgioItalia]",Roma,False,0,1677,NaN,[r_esistancee]
1411092100695875586,MeddyTeranee,Fri Jul 02 22:39:08 +0000 2021,RT @ManagerMitsuki: La définition de récolter ...,0,[BELITA],Neo-Tokyo 3,False,0,163,NaN,[ManagerMitsuki]
1411092105234108417,hoonestlyy,Fri Jul 02 22:39:09 +0000 2021,RT @SoftyLouis91: la francia e il belgio non h...,0,"[BelgioItalia, BELITA]",juventus’ girl,False,0,171,NaN,[SoftyLouis91]
1411092105309663233,Impresa_Energy,Fri Jul 02 22:39:09 +0000 2021,RT @BSchweinsteiger: .@bonucci_leo19 &amp; @ch...,0,"[experience, BELITA]",None,False,0,365,NaN,"[BSchweinsteiger, bonucci_leo19, chiellini]"


In [ ]:
#scarico l'ultimo stream
# df.to_csv(f'{scrap_belita}sc_twitter_end.csv') #salvo in csv BELITA
# df.to_csv(f'{scrap_itaesp}sc_twitter_end.csv') #salvo in csv ITAESP
df.to_csv(f'{scrap_itaen}sc_twitter_end.csv') #salvo in csv ITAEN

In [ ]:
#unisco tutti i csv

path = f'{scrap_belita}'
all_files = glob.glob(os.path.join(path, "sc_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.drop_duplicates(subset=['id'], inplace=True, ignore_index=False)
df_merged.insert(len(df_merged.columns), 'match', 'belita')
df_merged.to_csv(f'{staging_area}merged_belita.csv', index=False)
print(f'BELITA: {df_merged.shape[0]} records | {df_merged.shape[1]} columns')

path = f'{scrap_itaesp}'
all_files = glob.glob(os.path.join(path, "sc_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.drop_duplicates(subset=['id'], inplace=True, ignore_index=False)
df_merged.insert(len(df_merged.columns), 'match', 'itaesp')
df_merged.to_csv(f'{staging_area}merged_itaesp.csv', index=False)
print(f'ITAESP: {df_merged.shape[0]} records | {df_merged.shape[1]} columns')


path = f'{scrap_itaen}'
all_files = glob.glob(os.path.join(path, "sc_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.drop_duplicates(subset=['id'], inplace=True, ignore_index=False)
df_merged.insert(len(df_merged.columns), 'match', 'itaen')
print(f'ITAEN: {df_merged.shape[0]} records | {df_merged.shape[1]} columns')

path = f'{staging_area}'
all_files = glob.glob(os.path.join(path, "merged_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv(f'{staging_area}merge.csv', index=False)
print(f'TOT: {df_merged.shape[0]} records | {df_merged.shape[1]} columns')
print()
print(f'{df_merged.columns}')

BELITA: 220647 records | 13 columns
ITAESP: 119762 records | 13 columns
ITAEN: 580111 records | 13 columns
TOT: 920528 records | 13 columns

Index(['id', 'user', 'date', 'text', 'favorite_count', 'hashtags', 'location',
       'retweet', 'retweet_count', 'followers_count', 'in_reply_to_status_id',
       'match', 'user_mentions'],
      dtype='object')


##ETL

###Drop NaN

In [ ]:
#carico in df pandas
df = pd.read_csv(f'{staging_area}merge.csv')

In [ ]:
#diamo un occhio
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920528 entries, 0 to 920527
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     920527 non-null  object 
 1   user                   920526 non-null  object 
 2   date                   920526 non-null  object 
 3   text                   920526 non-null  object 
 4   favorite_count         920524 non-null  object 
 5   hashtags               920525 non-null  object 
 6   location               593576 non-null  object 
 7   retweet                920519 non-null  object 
 8   retweet_count          920520 non-null  object 
 9   followers_count        920519 non-null  float64
 10  in_reply_to_status_id  11965 non-null   float64
 11  match                  920528 non-null  object 
 12  user_mentions          920519 non-null  object 
dtypes: float64(2), object(11)
memory usage: 91.3+ MB


In [ ]:
df.sample(3)

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,match,user_mentions
43329,1414302337477320707,CalaveraDoctor,Sun Jul 11 19:15:28 +0000 2021,RT @heavnlysaturn: FEDERICO CHIESA FACCI SOGNARE COME SAI FARE TU #Euro2020Final #ItaliaInghilterra #ITAENG https://t.co/SxyX55SckA,0,"['Euro2020Final', 'ItaliaInghilterra', 'ITAENG']","Diriomo, pueblo de brujos",False,0,33.0,NaN,itaen,['heavnlysaturn']
458698,1414344530271342593,_lordyy_,Sun Jul 11 22:03:07 +0000 2021,RT @MelMaria05: Escucho una canción pero estoy 100% segura que esa no es Butter\n#BTS_Butter\n#Euro2020Final https://t.co/17veZ4NyZQ,0,"['BTS_Butter', 'Euro2020Final']",Saturno,False,0,91.0,NaN,itaen,['MelMaria05']
241207,1414324043814252549,Alex_ethio,Sun Jul 11 20:41:43 +0000 2021,#Chiesa of #Italy is something else!\nIt seems like it's coming Rome. \n\n#Euro2020Final \n#EURO2020,0,"['Chiesa', 'Italy', 'Euro2020Final', 'EURO2020']",Luxembourg,False,0,16.0,NaN,itaen,[]


In [ ]:
#controllo i nulli
df.isna().sum()

id                            1
user                          2
date                          2
text                          2
favorite_count                4
hashtags                      3
location                 326952
retweet                       9
retweet_count                 8
followers_count               9
in_reply_to_status_id    908563
match                         0
user_mentions                 9
dtype: int64

In [ ]:
# location e in_reply_to_status_id non mi servono
df.drop(['location','in_reply_to_status_id'], inplace=True, axis=1)
# elimino tutti i record che contengono almeno un NaN visto che sono pochi
df.dropna(inplace=True)
#elimino tutti i duplicati
df.isna().sum()

id                 0
user               0
date               0
text               0
favorite_count     0
hashtags           0
retweet            0
retweet_count      0
followers_count    0
match              0
user_mentions      0
dtype: int64

In [ ]:
#cosa rimane?
print(f"Number of records without duplicates and NaN {df.shape[0]} di 920528 records")

Number of records without duplicates and NaN 920519 di 920528 records


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 920519 entries, 0 to 920527
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               920519 non-null  object 
 1   user             920519 non-null  object 
 2   date             920519 non-null  object 
 3   text             920519 non-null  object 
 4   favorite_count   920519 non-null  object 
 5   hashtags         920519 non-null  object 
 6   retweet          920519 non-null  object 
 7   retweet_count    920519 non-null  object 
 8   followers_count  920519 non-null  float64
 9   match            920519 non-null  object 
 10  user_mentions    920519 non-null  object 
dtypes: float64(1), object(10)
memory usage: 84.3+ MB


In [ ]:
df.to_csv(f'{staging_area}merge_no_nan_01.csv', index=False, encoding="utf-8")

###Drop columns

In [ ]:
#creo altro df con ciò che mi serve
p_project = df[['id', 'user', 'date', 'text', 'hashtags', 'user_mentions', 'match']]
p_project.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 920519 entries, 0 to 920527
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             920519 non-null  object
 1   user           920519 non-null  object
 2   date           920519 non-null  object
 3   text           920519 non-null  object
 4   hashtags       920519 non-null  object
 5   user_mentions  920519 non-null  object
 6   match          920519 non-null  object
dtypes: object(7)
memory usage: 56.2+ MB


In [ ]:
#carico il file dopo una prima pulizia
p_project.to_csv(f'{staging_area}merge_drop_col_02.csv', index=False)

###Cleaning

In [ ]:
import csv
with open(f'{staging_area}merge_demoji_and_cleaned_03.csv', mode='w') as csv_file_out:
  csv_writer_out = csv.writer(csv_file_out, delimiter=',', quotechar='"')
  with open(f'{staging_area}merge_drop_col_02.csv', mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    max_lines=10000000 
    monDict = {'Gen' : '01' , 'Feb' : '02', 'Mar' : '03', 'Apr' : '04', 'May' : '05', 'Jun' : '06', 'Jul' : '07', 'Aug' : '08', 'Sep' : '09', 'Oct' : '10', 'Nov' : '11', 'Dic' : '12'}
  
    for row in csv_reader:
          if line_count > 0:
              dateCur=row[2]
              monCur=monDict[dateCur[4:7]]
              dateNew=dateCur[26:30]+'-'+monDict[dateCur[4:7]]+'-'+dateCur[8:10]+' '+dateCur[11:13]+':'+dateCur[14:16]+':'+dateCur[17:19]
              timezone='z'+dateCur[20:25]
              #print (f'{dateCur} ==> {dateNew}')
              textnonl=row[3].replace("\n"," ")
              textnonl=re.sub(r"\s+"," ",textnonl) # tolgo doppi spazi
              #print (f'message {message}')
              textnonlhash=re.sub(r"#\w+ ?","",textnonl)
              textnonlhash=re.sub(r"\s+"," ",textnonlhash)
              textnonlhashmen=re.sub(r"@\w+ ?","",textnonlhash)
              textnonlhashmen=re.sub(r"\s+"," ",textnonlhashmen)
              textnonlhashmenhttp=re.sub(r"http\S+","",textnonlhashmen)
              textnonlhashmenhttp=re.sub(r"\s+"," ",textnonlhashmenhttp)
              textnonlhashmenhttpnum=re.sub(r"\d+", "", textnonlhashmenhttp)    
              textnonlhashmenhttpnum=re.sub(r"\s+"," ",textnonlhashmenhttpnum)
              textnonlhashmenhttpnumsimb=re.sub(r"[^\w\s]"," ",textnonlhashmenhttpnum)
              textnonlhashmenhttpnumsimb=re.sub(r"\s+"," ",textnonlhashmenhttpnumsimb)
              row2=row
              row2.extend([dateNew,timezone,textnonl,textnonlhash,textnonlhashmen,textnonlhashmenhttp,textnonlhashmenhttpnum,textnonlhashmenhttpnumsimb])
              csv_writer_out.writerow(row2)
              line_count += 1
          else:
            row2=row
            row2.extend(['date_norm','time_zone','text_nonl','text_nonl_hash','text_nonl_hash_men','text_nonl_hash_men_http','text_nonl_hash_men_http_num','text_nonl_hash_men_http_num_simb'])
            csv_writer_out.writerow(row2)
            line_count += 1
          if line_count > max_lines:
              print (f'Exit')
              break      
    print(f'Processed {line_count} lines.')

Processed 920520 lines.


In [ ]:
df = pd.read_csv(f'{staging_area}merge_demoji_and_cleaned_03.csv', sep=',')
df.dropna(inplace=True)
df.to_csv(f'{staging_area}merge_demoji_and_cleaned_03.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908399 entries, 0 to 920518
Data columns (total 15 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                908399 non-null  float64
 1   user                              908399 non-null  object 
 2   date                              908399 non-null  object 
 3   text                              908399 non-null  object 
 4   hashtags                          908399 non-null  object 
 5   user_mentions                     908399 non-null  object 
 6   match                             908399 non-null  object 
 7   date_norm                         908399 non-null  object 
 8   time_zone                         908399 non-null  object 
 9   text_nonl                         908399 non-null  object 
 10  text_nonl_hash                    908399 non-null  object 
 11  text_nonl_hash_men                908399 non-null  o

In [ ]:
df.head()

,id,user,date,text,hashtags,user_mentions,match,date_norm,time_zone,text_nonl,text_nonl_hash,text_nonl_hash_men,text_nonl_hash_men_http,text_nonl_hash_men_http_num,text_nonl_hash_men_http_num_simb
0,1.414299e+18,Andrea6Fornara,Sun Jul 11 19:00:35 +0000 2021,"La macchina di Insigne porta la palla, inginocchiamento e si parte\n#Euro2020Final #ITAENG","['Euro2020Final', 'ITAENG']",[],itaen,2021-07-11 19:00:35,z+0000,"La macchina di Insigne porta la palla, inginocchiamento e si parte #Euro2020Final #ITAENG","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte",La macchina di Insigne porta la palla inginocchiamento e si parte
1,1.414299e+18,stanaccountsg,Sun Jul 11 19:01:42 +0000 2021,RT @___Ameliee: CHE INIZI L’AGONIA \n#Euro2020Final #EURO2020 #ItaliaInghilterra,"['Euro2020Final', 'EURO2020', 'ItaliaInghilterra']",['___Ameliee'],itaen,2021-07-11 19:01:42,z+0000,RT @___Ameliee: CHE INIZI L’AGONIA #Euro2020Final #EURO2020 #ItaliaInghilterra,RT @___Ameliee: CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT CHE INIZI L AGONIA
2,1.414299e+18,BR_buriren,Sun Jul 11 19:01:42 +0000 2021,決勝始まったー\nエドのためにもイングランドやっちゃいなー #Euro2020Final,['Euro2020Final'],[],itaen,2021-07-11 19:01:42,z+0000,決勝始まったー エドのためにもイングランドやっちゃいなー #Euro2020Final,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー
4,1.414299e+18,HeatherFidler2,Sun Jul 11 19:01:42 +0000 2021,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,[],[],itaen,2021-07-11 19:01:42,z+0000,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum I Love Bakewell Tarts Go England
5,1.414299e+18,Parksey1988,Sun Jul 11 19:01:42 +0000 2021,Really doesn't matter if we don't win this #Euro2020Final we made this final and that's the progression from hear on! #ITAENG,"['Euro2020Final', 'ITAENG']",[],itaen,2021-07-11 19:01:42,z+0000,Really doesn't matter if we don't win this #Euro2020Final we made this final and that's the progression from hear on! #ITAENG,Really doesn't matter if we don't win this we made this final and that's the progression from hear on!,Really doesn't matter if we don't win this we made this final and that's the progression from hear on!,Really doesn't matter if we don't win this we made this final and that's the progression from hear on!,Really doesn't matter if we don't win this we made this final and that's the progression from hear on!,Really doesn t matter if we don t win this we made this final and that s the progression from hear on


###Language detection

In [27]:
!pip install polyglot
!pip install PyICU
!pip install pycld2
!pip install morfessor
import polyglot
import icu
from polyglot.detect import Detector
from icu import *

In [28]:
p_project_cleaned = pd.read_csv(f'{staging_area}merge_demoji_and_cleaned_03.csv', sep=',')
p_project_cleaned.sample(3)

,id,user,date,text,hashtags,user_mentions,match,date_norm,time_zone,text_nonl,text_nonl_hash,text_nonl_hash_men,text_nonl_hash_men_http,text_nonl_hash_men_http_num,text_nonl_hash_men_http_num_simb
73613,1.414306e+18,gbrfrc__,Sun Jul 11 19:29:17 +0000 2021,RT @Una_Gioia_Mai: L'Italia ripudia la guerra ma fino a un certo punto\nQuesto non dovevate\n#Euro2020Final #ItaliaInghilterra #ITAENG https:…,"['Euro2020Final', 'ItaliaInghilterra', 'ITAENG']",['Una_Gioia_Mai'],itaen,2021-07-11 19:29:17,z+0000,RT @Una_Gioia_Mai: L'Italia ripudia la guerra ma fino a un certo punto Questo non dovevate #Euro2020Final #ItaliaInghilterra #ITAENG https:…,RT @Una_Gioia_Mai: L'Italia ripudia la guerra ma fino a un certo punto Questo non dovevate https:…,RT : L'Italia ripudia la guerra ma fino a un certo punto Questo non dovevate https:…,RT : L'Italia ripudia la guerra ma fino a un certo punto Questo non dovevate,RT : L'Italia ripudia la guerra ma fino a un certo punto Questo non dovevate,RT L Italia ripudia la guerra ma fino a un certo punto Questo non dovevate
734991,1.411068e+18,nraks_,Fri Jul 02 21:04:52 +0000 2021,RT @Powwwpy: Les italiens toutes les deux minutes #BELITA https://t.co/GvwLWFzbrX,['BELITA'],['Powwwpy'],belita,2021-07-02 21:04:52,z+0000,RT @Powwwpy: Les italiens toutes les deux minutes #BELITA https://t.co/GvwLWFzbrX,RT @Powwwpy: Les italiens toutes les deux minutes https://t.co/GvwLWFzbrX,RT : Les italiens toutes les deux minutes https://t.co/GvwLWFzbrX,RT : Les italiens toutes les deux minutes,RT : Les italiens toutes les deux minutes,RT Les italiens toutes les deux minutes
619785,1.411048e+18,ladylevia,Fri Jul 02 19:42:14 +0000 2021,RT @azzurri: #EURO2020 \n\n⏱️ 41' - #Chiesa curls one towards goal. Just wide. 😮\n\n#BELITA 0️⃣-1⃣\n\n#ITA #Azzurri #VivoAzzurro,"['EURO2020', 'Chiesa', 'BELITA', 'ITA', 'Azzurri', 'VivoAzzurro']",['azzurri'],belita,2021-07-02 19:42:14,z+0000,RT @azzurri: #EURO2020 ⏱️ 41' - #Chiesa curls one towards goal. Just wide. 😮 #BELITA 0️⃣-1⃣ #ITA #Azzurri #VivoAzzurro,RT @azzurri: ⏱️ 41' - curls one towards goal. Just wide. 😮 0️⃣-1⃣,RT : ⏱️ 41' - curls one towards goal. Just wide. 😮 0️⃣-1⃣,RT : ⏱️ 41' - curls one towards goal. Just wide. 😮 0️⃣-1⃣,RT : ⏱️ ' - curls one towards goal. Just wide. 😮 ️⃣-⃣,RT curls one towards goal Just wide


In [ ]:
# problema: servono almeno 3 parole per tweet altrimenti polyglot si rompe dopo un po

In [29]:
#nuova colonna con testo splittato
p_project_cleaned['split_text'] = p_project_cleaned['text_nonl_hash_men_http_num_simb'].str.split()
p_project_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908399 entries, 0 to 908398
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                908399 non-null  float64
 1   user                              908399 non-null  object 
 2   date                              908399 non-null  object 
 3   text                              908399 non-null  object 
 4   hashtags                          908399 non-null  object 
 5   user_mentions                     908399 non-null  object 
 6   match                             908399 non-null  object 
 7   date_norm                         908399 non-null  object 
 8   time_zone                         908399 non-null  object 
 9   text_nonl                         908399 non-null  object 
 10  text_nonl_hash                    908399 non-null  object 
 11  text_nonl_hash_men                908399 non-null  o

In [30]:
#nuova colonna con la conta delle parole
p_project_cleaned['length'] = p_project_cleaned['split_text'].apply(lambda x: len(x))

In [31]:
#vediamo
p_project_cleaned.head(3)

,id,user,date,text,hashtags,user_mentions,match,date_norm,time_zone,text_nonl,text_nonl_hash,text_nonl_hash_men,text_nonl_hash_men_http,text_nonl_hash_men_http_num,text_nonl_hash_men_http_num_simb,split_text,length
0,1.414299e+18,Andrea6Fornara,Sun Jul 11 19:00:35 +0000 2021,"La macchina di Insigne porta la palla, inginocchiamento e si parte\n#Euro2020Final #ITAENG","['Euro2020Final', 'ITAENG']",[],itaen,2021-07-11 19:00:35,z+0000,"La macchina di Insigne porta la palla, inginocchiamento e si parte #Euro2020Final #ITAENG","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte",La macchina di Insigne porta la palla inginocchiamento e si parte,"[La, macchina, di, Insigne, porta, la, palla, inginocchiamento, e, si, parte]",11
1,1.414299e+18,stanaccountsg,Sun Jul 11 19:01:42 +0000 2021,RT @___Ameliee: CHE INIZI L’AGONIA \n#Euro2020Final #EURO2020 #ItaliaInghilterra,"['Euro2020Final', 'EURO2020', 'ItaliaInghilterra']",['___Ameliee'],itaen,2021-07-11 19:01:42,z+0000,RT @___Ameliee: CHE INIZI L’AGONIA #Euro2020Final #EURO2020 #ItaliaInghilterra,RT @___Ameliee: CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT CHE INIZI L AGONIA,"[RT, CHE, INIZI, L, AGONIA]",5
2,1.414299e+18,BR_buriren,Sun Jul 11 19:01:42 +0000 2021,決勝始まったー\nエドのためにもイングランドやっちゃいなー #Euro2020Final,['Euro2020Final'],[],itaen,2021-07-11 19:01:42,z+0000,決勝始まったー エドのためにもイングランドやっちゃいなー #Euro2020Final,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー,決勝始まったー エドのためにもイングランドやっちゃいなー,"[決勝始まったー, エドのためにもイングランドやっちゃいなー]",2


In [32]:
#ci sono valori 0
min(p_project_cleaned['length'])

0

In [33]:
#tengo solo tweet con 3 parole almeno
p_project_cleaned = p_project_cleaned[p_project_cleaned.length > 2]

In [34]:
p_project_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828635 entries, 0 to 908398
Data columns (total 17 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                828635 non-null  float64
 1   user                              828635 non-null  object 
 2   date                              828635 non-null  object 
 3   text                              828635 non-null  object 
 4   hashtags                          828635 non-null  object 
 5   user_mentions                     828635 non-null  object 
 6   match                             828635 non-null  object 
 7   date_norm                         828635 non-null  object 
 8   time_zone                         828635 non-null  object 
 9   text_nonl                         828635 non-null  object 
 10  text_nonl_hash                    828635 non-null  object 
 11  text_nonl_hash_men                828635 non-null  o

In [35]:
#dividere il subset in 3 perchè la RAM di Colab non è sufficiente
p_project_cleaned_1 = p_project_cleaned.iloc[:100000,:]
p_project_cleaned_2 = p_project_cleaned.iloc[100000:200000,:]
p_project_cleaned_3 = p_project_cleaned.iloc[200000:300000,:]
p_project_cleaned_4 = p_project_cleaned.iloc[300000:400000,:]
p_project_cleaned_5 = p_project_cleaned.iloc[400000:500000,:]
p_project_cleaned_6 = p_project_cleaned.iloc[500000:600000,:]
p_project_cleaned_7 = p_project_cleaned.iloc[600000:700000,:]
p_project_cleaned_8 = p_project_cleaned.iloc[700000:800000,:]
p_project_cleaned_9 = p_project_cleaned.iloc[800000:,:]

In [36]:
# apply polyglot
p_project_cleaned_1['poly_obj'] = p_project_cleaned_1['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_1['text_lang'] = p_project_cleaned_1.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_1['text_lan_code'] = p_project_cleaned_1.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_1['lang_confidence'] = p_project_cleaned_1.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_1.to_csv(f'{staging_area}subset_lan_det_01.csv', index=False)

Output streaming troncato alle ultime 5000 righe.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect t

In [37]:
p_project_cleaned_2['poly_obj'] = p_project_cleaned_2['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_2['text_lang'] = p_project_cleaned_2.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_2['text_lan_code'] = p_project_cleaned_2.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_2['lang_confidence'] = p_project_cleaned_2.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_2.to_csv(f'{staging_area}subset_lan_det_02.csv', index=False)

p_project_cleaned_3['poly_obj'] = p_project_cleaned_3['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_3['text_lang'] = p_project_cleaned_3.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_3['text_lan_code'] = p_project_cleaned_3.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_3['lang_confidence'] = p_project_cleaned_3.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_3.to_csv(f'{staging_area}subset_lan_det_03.csv', index=False)

p_project_cleaned_4['poly_obj'] = p_project_cleaned_4['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_4['text_lang'] = p_project_cleaned_4.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_4['text_lan_code'] = p_project_cleaned_4.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_4['lang_confidence'] = p_project_cleaned_4.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_4.to_csv(f'{staging_area}subset_lan_det_04.csv', index=False)

p_project_cleaned_5['poly_obj'] = p_project_cleaned_5['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_5['text_lang'] = p_project_cleaned_5.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_5['text_lan_code'] = p_project_cleaned_5.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_5['lang_confidence'] = p_project_cleaned_5.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_5.to_csv(f'{staging_area}subset_lan_det_05.csv', index=False)

p_project_cleaned_6['poly_obj'] = p_project_cleaned_6['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_6['text_lang'] = p_project_cleaned_6.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_6['text_lan_code'] = p_project_cleaned_6.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_6['lang_confidence'] = p_project_cleaned_6.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_6.to_csv(f'{staging_area}subset_lan_det_06.csv', index=False)

Output streaming troncato alle ultime 5000 righe.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect t

In [38]:
p_project_cleaned_7['poly_obj'] = p_project_cleaned_7['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_7['text_lang'] = p_project_cleaned_7.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_7['text_lan_code'] = p_project_cleaned_7.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_7['lang_confidence'] = p_project_cleaned_7.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_7.to_csv(f'{staging_area}subset_lan_det_07.csv', index=False)

p_project_cleaned_8['poly_obj'] = p_project_cleaned_8['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_8['text_lang'] = p_project_cleaned_8.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_8['text_lan_code'] = p_project_cleaned_8.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_8['lang_confidence'] = p_project_cleaned_8.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_8.to_csv(f'{staging_area}subset_lan_det_08.csv', index=False)

p_project_cleaned_9['poly_obj'] = p_project_cleaned_9['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
p_project_cleaned_9['text_lang'] = p_project_cleaned_9.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
p_project_cleaned_9['text_lan_code'] = p_project_cleaned_9.poly_obj.apply( lambda x: x.language.code)
p_project_cleaned_9['lang_confidence'] = p_project_cleaned_9.poly_obj.apply( lambda x: x.language.confidence)
p_project_cleaned_9.to_csv(f'{staging_area}subset_lan_det_09.csv', index=False)

Output streaming troncato alle ultime 5000 righe.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect t

In [39]:
path = f'{staging_area}'
all_files = glob.glob(os.path.join(path, "subset_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv(f'{staging_area}merge_lan_det_04.csv', index=False)
print(f'TOT: {df_merged.shape[0]} records | {df_merged.shape[1]} columns')
print()
print(f'{df_merged.columns}')

TOT: 828635 records | 21 columns

Index(['id', 'user', 'date', 'text', 'hashtags', 'user_mentions', 'match',
       'date_norm', 'time_zone', 'text_nonl', 'text_nonl_hash',
       'text_nonl_hash_men', 'text_nonl_hash_men_http',
       'text_nonl_hash_men_http_num', 'text_nonl_hash_men_http_num_simb',
       'split_text', 'length', 'poly_obj', 'text_lang', 'text_lan_code',
       'lang_confidence'],
      dtype='object')


In [40]:
#vediamo
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828635 entries, 0 to 828634
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                828635 non-null  float64
 1   user                              828635 non-null  object 
 2   date                              828635 non-null  object 
 3   text                              828635 non-null  object 
 4   hashtags                          828635 non-null  object 
 5   user_mentions                     828635 non-null  object 
 6   match                             828635 non-null  object 
 7   date_norm                         828635 non-null  object 
 8   time_zone                         828635 non-null  object 
 9   text_nonl                         828635 non-null  object 
 10  text_nonl_hash                    828635 non-null  object 
 11  text_nonl_hash_men                828635 non-null  o

In [41]:
df_merged.head(3)

,id,user,date,text,hashtags,user_mentions,match,date_norm,time_zone,text_nonl,text_nonl_hash,text_nonl_hash_men,text_nonl_hash_men_http,text_nonl_hash_men_http_num,text_nonl_hash_men_http_num_simb,split_text,length,poly_obj,text_lang,text_lan_code,lang_confidence
0,1.414299e+18,Andrea6Fornara,Sun Jul 11 19:00:35 +0000 2021,"La macchina di Insigne porta la palla, inginocchiamento e si parte\n#Euro2020Final #ITAENG","['Euro2020Final', 'ITAENG']",[],itaen,2021-07-11 19:00:35,z+0000,"La macchina di Insigne porta la palla, inginocchiamento e si parte #Euro2020Final #ITAENG","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte",La macchina di Insigne porta la palla inginocchiamento e si parte,"['La', 'macchina', 'di', 'Insigne', 'porta', 'la', 'palla', 'inginocchiamento', 'e', 'si', 'parte']",11,Prediction is reliable: True\nLanguage 1: name: Italian code: it confidence: 98.0 read bytes: 310\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,Italian,it,98.0
1,1.414299e+18,stanaccountsg,Sun Jul 11 19:01:42 +0000 2021,RT @___Ameliee: CHE INIZI L’AGONIA \n#Euro2020Final #EURO2020 #ItaliaInghilterra,"['Euro2020Final', 'EURO2020', 'ItaliaInghilterra']",['___Ameliee'],itaen,2021-07-11 19:01:42,z+0000,RT @___Ameliee: CHE INIZI L’AGONIA #Euro2020Final #EURO2020 #ItaliaInghilterra,RT @___Ameliee: CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT CHE INIZI L AGONIA,"['RT', 'CHE', 'INIZI', 'L', 'AGONIA']",5,Prediction is reliable: False\nLanguage 1: name: Breton code: br confidence: 95.0 read bytes: 558\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,Breton,br,95.0
2,1.414299e+18,HeatherFidler2,Sun Jul 11 19:01:42 +0000 2021,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,[],[],itaen,2021-07-11 19:01:42,z+0000,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum 😋 I Love ❤️ Bakewell Tarts “Go England “🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽️,Oh yum I Love Bakewell Tarts Go England,"['Oh', 'yum', 'I', 'Love', 'Bakewell', 'Tarts', 'Go', 'England']",8,Prediction is reliable: False\nLanguage 1: name: English code: en confidence: 97.0 read bytes: 204\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,English,en,97.0


In [42]:
# vediamo che roba c'è dentro in una riga a caso
print(df_merged['poly_obj'][1])

Prediction is reliable: False
Language 1: name: Breton      code: br       confidence:  95.0 read bytes:   558
Language 2: name: un          code: un       confidence:   0.0 read bytes:     0
Language 3: name: un          code: un       confidence:   0.0 read bytes:     0


###Prepare pattern for sentiment

In [6]:
#riprendiamo
result_project = pd.read_csv(f'{staging_area}merge_lan_det_04.csv')

In [7]:
#definisco un pattern di insulti
mylist = ['cazzo', 'dio', 'madonna', 'merda', 'minchia', 'culo', 'stronzo', 'troia', 'figlio di', 'scarpone', 'puttana', 'cornuto', 'porco', 'cagare', 'coglione', 'incazzare', 'figa', 'bastardo', 'venduto', 'cazzata', 'stronzata', 'negro', 'vacca', 'pirla', 'frocio', 'pippa', 'terrone', 'suca', 'coleroso', 'tumorato']
pattern = '|'.join(mylist)
pattern
'(?i)cazzo|(?i)dio|(?i)madonna|(?i)merda|(?i)minchia|(?i)culo|(?i)stronzo|(?i)troia|(?i)figlio di|(?i)scarpone|(?i)puttana|(?i)cornuto|(?i)porco|(?i)cagare|(?i)coglione|(?i)incazzare|(?i)figa|(?i)bastardo|(?i)venduto|(?i)cazzata|(?i)stronzata|(?i)negro|(?i)vacca|(?i)pirla|(?i)frocio|(?i)pippa|(?i)terrone|(?i)suca|(?i)coleroso|(?i)tumorato'

'(?i)cazzo|(?i)dio|(?i)madonna|(?i)merda|(?i)minchia|(?i)culo|(?i)stronzo|(?i)troia|(?i)figlio di|(?i)scarpone|(?i)puttana|(?i)cornuto|(?i)porco|(?i)cagare|(?i)coglione|(?i)incazzare|(?i)figa|(?i)bastardo|(?i)venduto|(?i)cazzata|(?i)stronzata|(?i)negro|(?i)vacca|(?i)pirla|(?i)frocio|(?i)pippa|(?i)terrone|(?i)suca|(?i)coleroso|(?i)tumorato'

In [8]:
#creo una nuova colonna che mi dice se il tweet contiene o no qualcosa del pattern
result_project['insulto_ita'] = result_project.text_nonl_hash_men_http_num_simb.str.contains(pattern)

In [9]:
#vediamo
result_project.head(2)

,id,user,date,text,hashtags,user_mentions,match,date_norm,time_zone,text_nonl,text_nonl_hash,text_nonl_hash_men,text_nonl_hash_men_http,text_nonl_hash_men_http_num,text_nonl_hash_men_http_num_simb,split_text,length,poly_obj,text_lang,text_lan_code,lang_confidence,insulto_ita
0,1.414299e+18,Andrea6Fornara,Sun Jul 11 19:00:35 +0000 2021,"La macchina di Insigne porta la palla, inginocchiamento e si parte\n#Euro2020Final #ITAENG","['Euro2020Final', 'ITAENG']",[],itaen,2021-07-11 19:00:35,z+0000,"La macchina di Insigne porta la palla, inginocchiamento e si parte #Euro2020Final #ITAENG","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte",La macchina di Insigne porta la palla inginocchiamento e si parte,"['La', 'macchina', 'di', 'Insigne', 'porta', 'la', 'palla', 'inginocchiamento', 'e', 'si', 'parte']",11,Prediction is reliable: True\nLanguage 1: name: Italian code: it confidence: 98.0 read bytes: 310\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,Italian,it,98.0,False
1,1.414299e+18,stanaccountsg,Sun Jul 11 19:01:42 +0000 2021,RT @___Ameliee: CHE INIZI L’AGONIA \n#Euro2020Final #EURO2020 #ItaliaInghilterra,"['Euro2020Final', 'EURO2020', 'ItaliaInghilterra']",['___Ameliee'],itaen,2021-07-11 19:01:42,z+0000,RT @___Ameliee: CHE INIZI L’AGONIA #Euro2020Final #EURO2020 #ItaliaInghilterra,RT @___Ameliee: CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT : CHE INIZI L’AGONIA,RT CHE INIZI L AGONIA,"['RT', 'CHE', 'INIZI', 'L', 'AGONIA']",5,Prediction is reliable: False\nLanguage 1: name: Breton code: br confidence: 95.0 read bytes: 558\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,Breton,br,95.0,False


In [10]:
result_project.to_csv(f'{staging_area}merge_insult_05.csv', index=False)

##Analysis

In [4]:
#importo file
p = pd.read_csv(f'{staging_area}merge_insult_05.csv')

In [5]:
#divido in 3 subset per ogni match perchè Gepi fa schifo
belita = p[p["match"] == 'belita']
itaesp = p[p["match"] == 'itaesp']
itaen = p[p["match"] == 'itaen']

In [13]:
#conto gli insulti in lingua italiana, tutti, anche quelli che hanno prediction della lingua false
p['insulto_ita'].value_counts()

False    808806
True      19829
Name: insulto_ita, dtype: int64

In [14]:
#faccio un groupby sulle lingue e una count per sapere quante lingue ci sono e quale è la frequenza
language = p.groupby(['text_lang']).size()
language

text_lang
Abkhazian      2
Afar         139
Afrikaans    392
Akan          49
Albanian      63
            ... 
Zhuang        80
Zulu          63
bh            26
un           596
zzp          123
Length: 148, dtype: int64

In [15]:
#esporto quanto sopra così ci posso lavorare in google sheet e fare eventuali grafici
belita.to_excel(f'{dwh_euro2020}belita.xlsx')
itaesp.to_excel(f'{dwh_euro2020}itaesp.xlsx')
itaen.to_excel(f'{dwh_euro2020}itaen.xlsx')

In [16]:
#creo un subset solo di tweet in italiano
belita = p.loc[p['text_lang'] == 'Italian']
belita.head()

,id,user,date,text,hashtags,user_mentions,match,date_norm,time_zone,text_nonl,text_nonl_hash,text_nonl_hash_men,text_nonl_hash_men_http,text_nonl_hash_men_http_num,text_nonl_hash_men_http_num_simb,split_text,length,poly_obj,text_lang,text_lan_code,lang_confidence,insulto_ita
0,1.414299e+18,Andrea6Fornara,Sun Jul 11 19:00:35 +0000 2021,"La macchina di Insigne porta la palla, inginocchiamento e si parte\n#Euro2020Final #ITAENG","['Euro2020Final', 'ITAENG']",[],itaen,2021-07-11 19:00:35,z+0000,"La macchina di Insigne porta la palla, inginocchiamento e si parte #Euro2020Final #ITAENG","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte","La macchina di Insigne porta la palla, inginocchiamento e si parte",La macchina di Insigne porta la palla inginocchiamento e si parte,"['La', 'macchina', 'di', 'Insigne', 'porta', 'la', 'palla', 'inginocchiamento', 'e', 'si', 'parte']",11,Prediction is reliable: True\nLanguage 1: name: Italian code: it confidence: 98.0 read bytes: 310\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,Italian,it,98.0,False
8,1.414299e+18,gabrielobeloo,Sun Jul 11 19:01:42 +0000 2021,"Lui non voleva essere lì stasera, l’hanno costretto #ITAENG #ItaliaInghilterra #Euro2020final https://t.co/nbYt6WVpM6","['ITAENG', 'ItaliaInghilterra', 'Euro2020final']",[],itaen,2021-07-11 19:01:42,z+0000,"Lui non voleva essere lì stasera, l’hanno costretto #ITAENG #ItaliaInghilterra #Euro2020final https://t.co/nbYt6WVpM6","Lui non voleva essere lì stasera, l’hanno costretto https://t.co/nbYt6WVpM6","Lui non voleva essere lì stasera, l’hanno costretto https://t.co/nbYt6WVpM6","Lui non voleva essere lì stasera, l’hanno costretto","Lui non voleva essere lì stasera, l’hanno costretto",Lui non voleva essere lì stasera l hanno costretto,"['Lui', 'non', 'voleva', 'essere', 'lì', 'stasera', 'l', 'hanno', 'costretto']",9,Prediction is reliable: True\nLanguage 1: name: Italian code: it confidence: 98.0 read bytes: 1417\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,Italian,it,98.0,False
9,1.414299e+18,poositiveviibez,Sun Jul 11 19:01:42 +0000 2021,RT @yleniaindenial: A quelli che hanno fischiato il nostro inno multa e dissenteria per una settimana \n\n#ItaliaInghilterra #Euro2020Final,"['ItaliaInghilterra', 'Euro2020Final']",['yleniaindenial'],itaen,2021-07-11 19:01:42,z+0000,RT @yleniaindenial: A quelli che hanno fischiato il nostro inno multa e dissenteria per una settimana #ItaliaInghilterra #Euro2020Final,RT @yleniaindenial: A quelli che hanno fischiato il nostro inno multa e dissenteria per una settimana,RT : A quelli che hanno fischiato il nostro inno multa e dissenteria per una settimana,RT : A quelli che hanno fischiato il nostro inno multa e dissenteria per una settimana,RT : A quelli che hanno fischiato il nostro inno multa e dissenteria per una settimana,RT A quelli che hanno fischiato il nostro inno multa e dissenteria per una settimana,"['RT', 'A', 'quelli', 'che', 'hanno', 'fischiato', 'il', 'nostro', 'inno', 'multa', 'e', 'dissenteria', 'per', 'una', 'settimana']",15,Prediction is reliable: True\nLanguage 1: name: Italian code: it confidence: 98.0 read bytes: 578\nLanguage 2: name: un code: un confidence: 0.0 read bytes: 0\nLanguage 3: name: un code: un confidence: 0.0 read bytes: 0,Italian,it,98.0,False
11,1.414299e+18,ilgrandevecchio,Sun Jul 11 19:01:42 +0000 2021,E comunque meno gente ora in giro che per la zona rossa #Euro2020Final,['Euro2020Final'],[],itaen,2021-07-11 19:01:42,z+0000,E comunque meno gente ora in giro che per la zona rossa #Euro2020Final,E comunque meno gente ora in giro che per la zona rossa,E comunque meno gente ora in giro che per la zona rossa,E comunque meno gente ora in giro che per la zona rossa,E comunque 

In [ ]:
###### DAI QUI IN POI, RISULTATO PER RISULTATO, COPIAVO A MANO SU GOOGLESHEET
### 22 GIOCATORI + VAR E ARBITRO + 2 ALLENATORI = 26 PERSONE DA TROVARE
# FATTIBILE FARE I PASSAGGI A MANO MA FOSSERO STATE MIGLIAIA DI PERSONE DA ANALIZZARE?
# CI IMPIEGHEREI TROPPO TEMPO A CAPIRE COME FARE E HO TAGLIATO CORTO
# MAGARI LA SOLUZIONE è PIù semplice dle previsto. amen

In [17]:
#conto gli insulti in lingua italiana nei tweet che risultano in italiano partendo dal subset precedente
belita['insulto_ita'].value_counts()

False    224518
True      14309
Name: insulto_ita, dtype: int64

In [18]:
#conto gli insulti in lingua italiana nei tweet che risultano in italiano dove la predizione è vera
#cioè l'assegnazione della lingua è data per certa
insult_it = belita.loc[belita['insulto_ita'] == True] #creo subset di insulti true da language ita
len(insult_it) #conto

14309

In [19]:
# controllo quante volte viene citato qualcuno in qualsiasi tweet
belita['text'].str.contains('ourtois').value_counts()

False    238801
True         26
Name: text, dtype: int64

In [20]:
# selezionio il subset delle volte in cui viene citato il giocatore in tutti i tweet con qualsiasi predizione
insulti = belita.loc[belita['text'].str.contains('ourtois')]

In [21]:
#controllo quante volte è stato insultato in italiano in tutti i tweet di tutte le lingue con qualsiasi predizione
insulti['insulto_ita'].value_counts()

False    26
Name: insulto_ita, dtype: int64

In [ ]:
#ora restringo l'anasili a citazioni e insulti solo per tweet con predizione true e in lingua italiana

In [28]:
# riprendo il sub set linguaggio in italiano e seleziono solo le volte in cui qualcuno è citato in tweet che risultano con prediction true
insult_it = insulti.loc[insulti['text'].str.contains('ourtois')]
insult_it['text'].value_counts()
#vedo che ci sono 14 + 1 retweet. non distingue e metto tutto insieme

RT @Vivo_Azzurro: #EURO2020\n\n⏱️ 27'\nTentativo di #Chiesa, Courtois non si fa sorprendere dalla deviazione di un compagno!\n\n🇧🇪🇮🇹 #BelgioItal…      14
“I due migliori n.1 al mondo: courtois e gigio donnarumma” \n\n #BELITA #euro2020 https://t.co/9X5wXkNYGb                                               1
Anche su Marte conoscono il tiraggir di Insigne, tranne Courtois. #BelgioItalia #BELITA #EURO2020                                                       1
Secondo me è che Barella e Insigne sono troppo bassi e Courtois non li vede. Al prossimo punto su Verratti #BELITA                                      1
RT @adiafora_: Secondo me è che Barella e Insigne sono troppo bassi e Courtois non li vede. Al prossimo punto su Verratti #BELITA                       1
#EURO2020\n\n⏱️ 27'\nTentativo di #Chiesa, Courtois non si fa sorprendere dalla deviazione di un compagno!\n\n🇧🇪🇮🇹… https://t.co/KFxsR7KDMr             1
...#Donnarumma stasera partitone. Ha stravinto il duello con #Courtois!\n\n#

In [29]:
len(insult_it) #conto

26

In [30]:
# controllo quante volte viene insultato in italiano il citato solo nei tweet sopra (in ling ita e validati)
insult_it['insulto_ita'].value_counts()

False    26
Name: insulto_ita, dtype: int64